In [1]:
#pip install cassandra-driver

In [2]:
# Cassandra database connection
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [4]:
import os
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Ensure the file paths are correctly specified
secure_bundle_path = r'C:\Users\pramod\Desktop\INURON\Internship\Insurance_Premium_Prediction\research\secure-connect-insurance.zip'
token_path = r'C:\Users\pramod\Desktop\INURON\Internship\Insurance_Premium_Prediction\research\Insurance-token.json'

# Set environment variables in your Python environment if necessary
os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"] = secure_bundle_path
os.environ["ASTRADB_APPLICATION_TOKEN"] = token_path

# Cloud configuration for the Cassandra cluster
cloud_config = {
    'secure_connect_bundle': os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"]
}

# Load the token JSON file
with open(os.environ["ASTRADB_APPLICATION_TOKEN"]) as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

# Set up authentication and connect to the cluster
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Execute a query to test the connection
row = session.execute("SELECT release_version FROM system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")


4.0.0.6816


In [14]:

# Set keyspace
session.set_keyspace('data_storage')

In [15]:
import pandas as pd

In [17]:
df = pd.read_csv("insurance.csv")
df.head(3)

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46


In [10]:

# Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS insurance_records (
    id UUID PRIMARY KEY,
    age INT,
    sex TEXT,
    bmi FLOAT,
    children INT,
    smoker TEXT,
    region TEXT,
    expenses FLOAT
)
"""
session.execute(create_table_query)


In [24]:
# Query to list all tables in the keyspace
tables_query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'data_storage';"
tables = session.execute(tables_query)

print("Tables in the keyspace:")
for table in tables:
    print(table.table_name)



Tables in the keyspace:
insurance_records


In [23]:


# Query to list all tables in the keyspace
tables_query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'data_storage';"
tables = session.execute(tables_query)

print("Tables in the keyspace:")
for table in tables:
    table_name = table.table_name
    print(f"Table: {table_name}")
    
    # Query to get the column names and types
    schema_query = f"""
    SELECT column_name, type
    FROM system_schema.columns
    WHERE keyspace_name = 'data_storage' AND table_name = '{table_name}';
    """
    columns = session.execute(schema_query)
    
    print("Columns:")
    column_names = []
    for column in columns:
        column_names.append(f"{column.column_name} ({column.type})")
    
    print(" | ".join(column_names))
    
    # Query to select data from the table
    select_query = f"SELECT * FROM {table_name} LIMIT 5;"
    rows = session.execute(select_query)
    
    print("Data:")
    for row in rows:
        print(row)
    print("\n" + "-"*50 + "\n")


Tables in the keyspace:
Table: insurance_records
Columns:
age (int) | bmi (float) | children (int) | expenses (float) | id (uuid) | region (text) | sex (text) | smoker (text)
Data:

--------------------------------------------------



In [27]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json


# Read CSV file into DataFrame
csv_file = 'insurance.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_file)

# Prepare insert statement
insert_stmt = session.prepare("""
    INSERT INTO insurance_records (id, age, sex, bmi, children, smoker, region, expenses)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""")

# Insert data into the table
from uuid import uuid4

for index, row in df.iterrows():
    session.execute(insert_stmt, (uuid4(), row['age'], row['sex'], row['bmi'], row['children'], row['smoker'], row['region'], row['expenses']))

print("Data inserted successfully.")


Data inserted successfully.


In [28]:
#use data_storage 
# SELECT * FROM insurance_records LIMIT 5;

In [37]:
rows = session.execute("select * from insurance_records limit 5")
for row in rows:
    print(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])


bfed2eec-a04f-4e5c-93d8-956d1b9161d7 47 32.0 1 8551.349609375 southwest female no
43b7df5a-b4f2-4eee-b1d8-71c3ec76bb0f 18 24.100000381469727 1 2201.10009765625 southeast female no
d727516f-7b80-4b31-8479-306fcd073f09 50 27.399999618530273 0 25656.580078125 northeast female no
95c790c4-125b-464f-ac34-12312441e015 57 29.0 0 27218.439453125 northeast male yes
f3c1a491-3296-4736-b781-440c01493924 54 29.200000762939453 1 10436.099609375 southwest male no


In [41]:
df[df["expenses"] >= 2201 ].sort_values(by = ['expenses']).head(2)

,age,sex,bmi,children,smoker,region,expenses
469,18,female,24.1,1,no,southeast,2201.10
471,18,female,30.1,0,no,northeast,2203.47


In [42]:
df.shape

(1338, 7)